# Data Processor

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

from datetime import datetime
import pytz

import re
import numpy as np

In [2]:
def str2int(num):
    try:
        return int(num)
    except:
        return 0

In [3]:
LAST_DATE = 'Sep 18, 2021'

# 1. VN Cases Data

In [4]:
# =================== GET HTML FROM URL ===================

url = "https://www.worldometers.info/coronavirus/country/viet-nam/"
r = requests.get(url)
html_text = r.text
    
soup = BeautifulSoup(html_text)
script_list = []
for tag in soup.find_all('script'):
    if tag.string != None and 'Highcharts.chart' in tag.string:
        script_list.append(tag.string)

        
# =================== EXTRACT DATA FROM HTML ===================

data_all = {}

for script in script_list:
    
    # extract data in each category
    data = {}

    pos = script.find('name:') + 7
    if pos - 7 == -1: continue
    pos2 = script.find("'", pos)
    name = script[pos:pos2].replace(' ', '')
    if name == 'DailyDeaths': continue
    
    pos = script.find('categories:') + 14
    pos2 = script.find(']', pos) - 1
    data['labels'] = script[pos:pos2].split('","')
    
    pos = script.find('data:') + 7
    pos2 = script.find(']', pos)
    data['data'] = list(map(str2int, script[pos:pos2].split(',')))
    
    data_all[name] = data
    
    # extract 7 days average
    if name == 'DailyCases':
        
        data2 = {}
        data2['labels'] = data['labels']
        
        pos = script.find("name: '7-day moving average'")
        pos = script.find('data:', pos) + 7
        pos2 = script.find(']', pos)
        data2['data'] = list(map(str2int, script[pos:pos2].split(',')))
        
        data_all['DailyCases7DaysAvg'] = data2
        
# calculate total recoveries
data = {'labels': data_all['Deaths']['labels']}
data['data'] = [data_all['Cases']['data'][i]-data_all['CurrentlyInfected']['data'][i]-data_all['Deaths']['data'][i] \
                for i in range(len(data_all['Deaths']['data']))]
data_all['Recoveries'] = data
del data_all['Cases']

print('Keys in data_all:')
for key in data_all:
    print(key)
print()


# =================== PROCESS DATE LABELS ===================

# check if all labels are the same
print('First, Last, Length of labels:')
for data in data_all.values():
    print(data['labels'][0], data['labels'][-1], len(data['labels']))

# convert to utc timestamp
index = data_all['DailyCases']['labels'].index(LAST_DATE)
print(data_all['DailyCases']['labels'][:index+1][-1], end='\n\n')
labels = list(map(lambda time_str: int(pytz.utc.localize(datetime.strptime(time_str, '%b %d, %Y')).timestamp()*1000),
                  data_all['DailyCases']['labels'][:index+1]))

# add missing dates, following world data
MISSINGS = [1579651200000, 1579737600000, 1579824000000, 1579910400000, 1579996800000, 1580083200000, 1580169600000, 1580256000000, 1580342400000, 1580428800000, 1580515200000, 1580601600000, 1580688000000, 1580774400000, 1580860800000, 1580947200000, 1581033600000, 1581120000000, 1581206400000, 1581292800000, 1581379200000, 1581465600000, 1581552000000, 1581638400000]
N_MISSINGS = len(MISSINGS)
labels = MISSINGS + labels


# =================== CONVERT TO JS FORMAT & SAVE TO FILE ===================

# convert to js format
js_text = 'const casesData = {\n  src: "https://www.worldometers.info/coronavirus/country/viet-nam/",\n'
for name, data in data_all.items():
    js_text += f'  {name}: ['
    js_text += ','.join([f'[{labels[i]},0]' for i in range(N_MISSINGS)])
    js_text += ','
    js_text += ','.join([f'[{labels[i+N_MISSINGS]},{data["data"][i]}]' for i in range(index+1)])
    js_text += '],\n'
js_text += '};'
js_text

# save to file
with open("processed/cases-data.js", "w") as fo:
    fo.write(js_text)
print('VN data saved successfully')

Keys in data_all:
DailyCases
DailyCases7DaysAvg
CurrentlyInfected
Deaths
Recoveries

First, Last, Length of labels:
Feb 15, 2020 Sep 20, 2021 584
Feb 15, 2020 Sep 20, 2021 584
Feb 15, 2020 Sep 20, 2021 584
Feb 15, 2020 Sep 20, 2021 584
Feb 15, 2020 Sep 20, 2021 584
Sep 18, 2021

VN data saved successfully


# 2. World & Foreign Countries Cases Data

In [5]:
URLS = [
    'https://www.worldometers.info/coronavirus/',
    'https://www.worldometers.info/coronavirus/country/us',
    'https://www.worldometers.info/coronavirus/country/china',
    'https://www.worldometers.info/coronavirus/country/india'
]
NAMES = ['world', 'usa', 'china', 'india']

MISSINGS = [1579651200000, 1579737600000, 1579824000000, 1579910400000, 1579996800000, 1580083200000, 1580169600000, 1580256000000, 1580342400000, 1580428800000, 1580515200000, 1580601600000, 1580688000000, 1580774400000, 1580860800000, 1580947200000, 1581033600000, 1581120000000, 1581206400000, 1581292800000, 1581379200000, 1581465600000, 1581552000000, 1581638400000]
N_MISSINGS = len(MISSINGS)

In [6]:
for global_i in range(len(URLS)):
    
    # =================== GET HTML FROM URL ===================
    print('==== ' + NAMES[global_i].title() + ' ====')

    url = URLS[global_i]
    r = requests.get(url)
    html_text = r.text

    soup = BeautifulSoup(html_text)
    script_list = []
    for tag in soup.find_all('script'):
        if tag.string != None and 'Highcharts.chart' in tag.string:
            script_list.append(tag.string)

    
    # =================== EXTRACT DATA FROM HTML ===================

    data_all = {}

    for script in script_list:

        # find the daily cases chart
        pos_chart = script.find('Daily New Cases')
        if pos_chart == -1: continue

        # extract daily cases
        data = {}

        pos = script.find('categories:', pos_chart) + 14
        pos2 = script.find(']', pos) - 1
        data['labels'] = script[pos:pos2].split('","')

        pos = script.find('data:', pos_chart) + 7
        pos2 = script.find(']', pos)
        data['data'] = list(map(str2int, script[pos:pos2].split(',')))

        data_all['DailyCases'] = data

        # extract 7 days average
        data2 = {}
        data2['labels'] = data['labels']

        pos = script.find("name: '7-day moving average'", pos_chart)
        pos = script.find('data:', pos) + 7
        pos2 = script.find(']', pos)
        data2['data'] = list(map(str2int, script[pos:pos2].split(',')))

        data_all['DailyCases7DaysAvg'] = data2

    for key in data_all:
        print(key)
        
        
    # =================== PROCESS DATE LABELS ===================
    
    # convert to utc timestamp
    index = data_all['DailyCases']['labels'].index(LAST_DATE)
    print(data_all['DailyCases']['labels'][-1], data_all['DailyCases']['labels'][:index+1][-1])
    labels = list(map(lambda time_str: int(pytz.utc.localize(datetime.strptime(time_str, '%b %d, %Y')).timestamp()*1000),
                      data_all['DailyCases']['labels'][:index+1]))
    
    # add missing dates, following world data
    if NAMES[global_i] in ('usa', 'india'):
        labels = MISSINGS + labels
    
    
    # =================== CONVERT TO JS FORMAT & SAVE TO FILE ===================
    
    # convert to js format
    js_text = 'const casesData' + NAMES[global_i].title() + ' = {\n  src: "' + URLS[global_i] + '",\n'
    for name, data in data_all.items():
        js_text += f'  {name}: ['
        if NAMES[global_i] in ('usa', 'india'):
            js_text += ','.join([f'[{labels[i]},0]' for i in range(N_MISSINGS)])
            js_text += ','
            js_text += ','.join([f'[{labels[i+N_MISSINGS]},{data["data"][i]}]' for i in range(index+1)])
        else:
            js_text += ','.join([f'[{labels[i]},{data["data"][i]}]' for i in range(index+1)])
        js_text += '],\n'
    js_text += '};'
    js_text

    # save to file
    with open(f"processed/cases-data-{NAMES[global_i]}.js", "w") as fo:
        fo.write(js_text)
    print(f'{NAMES[global_i].title()} data saved successfully', end='\n\n')

==== World ====
DailyCases
DailyCases7DaysAvg
Sep 20, 2021 Sep 18, 2021
World data saved successfully

==== Usa ====
DailyCases
DailyCases7DaysAvg
Sep 20, 2021 Sep 18, 2021
Usa data saved successfully

==== China ====
DailyCases
DailyCases7DaysAvg
Sep 20, 2021 Sep 18, 2021
China data saved successfully

==== India ====
DailyCases
DailyCases7DaysAvg
Sep 20, 2021 Sep 18, 2021
India data saved successfully



# 3. Vaccine Data

In [7]:
# source: https://ourworldindata.org/covid-vaccinations?country=VNM
#      => https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv

# =================== FILTER FOR VNM & PROCESS DATA ===================

# df = pd.read_csv('raw/vaccinations.csv', usecols=['iso_code', 'date', 'people_vaccinated', 'people_fully_vaccinated'])
df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv',
                 usecols=['iso_code', 'date', 'people_vaccinated', 'people_fully_vaccinated'])
df = df[df['iso_code'] == 'VNM']
df.drop('iso_code', axis=1, inplace=True)

# impute missing data & reformat
df['people_vaccinated'].interpolate(inplace=True)
df['people_fully_vaccinated'].interpolate(inplace=True)
df['people_fully_vaccinated'].fillna(0, inplace=True)
df['people_vaccinated'] = df['people_vaccinated'].astype('int32')
df['people_fully_vaccinated'] = df['people_fully_vaccinated'].astype('int32')
df['date'] = pd.to_datetime(df['date'])

# trim at LAST_DATE
index = df[df['date'] == LAST_DATE].index[0]
df = df.iloc[:index+1-df.index[0]]

# calculate percent of population
POPULATION = 98303150  # https://danso.org/viet-nam/
df['people_vaccinated_pct'] = df['people_vaccinated'] / POPULATION * 100
df['people_fully_vaccinated_pct'] = df['people_fully_vaccinated'] / POPULATION * 100


# =================== CONVERT DATETIME & SAVE TO FILE ===================

np.set_printoptions(threshold=np.inf)

MISSINGS = [1579651200000, 1579737600000, 1579824000000, 1579910400000, 1579996800000, 1580083200000, 1580169600000, 1580256000000, 1580342400000, 1580428800000, 1580515200000, 1580601600000, 1580688000000, 1580774400000, 1580860800000, 1580947200000, 1581033600000, 1581120000000, 1581206400000, 1581292800000, 1581379200000, 1581465600000, 1581552000000, 1581638400000, 1581724800000, 1581811200000, 1581897600000, 1581984000000, 1582070400000, 1582156800000, 1582243200000, 1582329600000, 1582416000000, 1582502400000, 1582588800000, 1582675200000, 1582761600000, 1582848000000, 1582934400000, 1583020800000, 1583107200000, 1583193600000, 1583280000000, 1583366400000, 1583452800000, 1583539200000, 1583625600000, 1583712000000, 1583798400000, 1583884800000, 1583971200000, 1584057600000, 1584144000000, 1584230400000, 1584316800000, 1584403200000, 1584489600000, 1584576000000, 1584662400000, 1584748800000, 1584835200000, 1584921600000, 1585008000000, 1585094400000, 1585180800000, 1585267200000, 1585353600000, 1585440000000, 1585526400000, 1585612800000, 1585699200000, 1585785600000, 1585872000000, 1585958400000, 1586044800000, 1586131200000, 1586217600000, 1586304000000, 1586390400000, 1586476800000, 1586563200000, 1586649600000, 1586736000000, 1586822400000, 1586908800000, 1586995200000, 1587081600000, 1587168000000, 1587254400000, 1587340800000, 1587427200000, 1587513600000, 1587600000000, 1587686400000, 1587772800000, 1587859200000, 1587945600000, 1588032000000, 1588118400000, 1588204800000, 1588291200000, 1588377600000, 1588464000000, 1588550400000, 1588636800000, 1588723200000, 1588809600000, 1588896000000, 1588982400000, 1589068800000, 1589155200000, 1589241600000, 1589328000000, 1589414400000, 1589500800000, 1589587200000, 1589673600000, 1589760000000, 1589846400000, 1589932800000, 1590019200000, 1590105600000, 1590192000000, 1590278400000, 1590364800000, 1590451200000, 1590537600000, 1590624000000, 1590710400000, 1590796800000, 1590883200000, 1590969600000, 1591056000000, 1591142400000, 1591228800000, 1591315200000, 1591401600000, 1591488000000, 1591574400000, 1591660800000, 1591747200000, 1591833600000, 1591920000000, 1592006400000, 1592092800000, 1592179200000, 1592265600000, 1592352000000, 1592438400000, 1592524800000, 1592611200000, 1592697600000, 1592784000000, 1592870400000, 1592956800000, 1593043200000, 1593129600000, 1593216000000, 1593302400000, 1593388800000, 1593475200000, 1593561600000, 1593648000000, 1593734400000, 1593820800000, 1593907200000, 1593993600000, 1594080000000, 1594166400000, 1594252800000, 1594339200000, 1594425600000, 1594512000000, 1594598400000, 1594684800000, 1594771200000, 1594857600000, 1594944000000, 1595030400000, 1595116800000, 1595203200000, 1595289600000, 1595376000000, 1595462400000, 1595548800000, 1595635200000, 1595721600000, 1595808000000, 1595894400000, 1595980800000, 1596067200000, 1596153600000, 1596240000000, 1596326400000, 1596412800000, 1596499200000, 1596585600000, 1596672000000, 1596758400000, 1596844800000, 1596931200000, 1597017600000, 1597104000000, 1597190400000, 1597276800000, 1597363200000, 1597449600000, 1597536000000, 1597622400000, 1597708800000, 1597795200000, 1597881600000, 1597968000000, 1598054400000, 1598140800000, 1598227200000, 1598313600000, 1598400000000, 1598486400000, 1598572800000, 1598659200000, 1598745600000, 1598832000000, 1598918400000, 1599004800000, 1599091200000, 1599177600000, 1599264000000, 1599350400000, 1599436800000, 1599523200000, 1599609600000, 1599696000000, 1599782400000, 1599868800000, 1599955200000, 1600041600000, 1600128000000, 1600214400000, 1600300800000, 1600387200000, 1600473600000, 1600560000000, 1600646400000, 1600732800000, 1600819200000, 1600905600000, 1600992000000, 1601078400000, 1601164800000, 1601251200000, 1601337600000, 1601424000000, 1601510400000, 1601596800000, 1601683200000, 1601769600000, 1601856000000, 1601942400000, 1602028800000, 1602115200000, 1602201600000, 1602288000000, 1602374400000, 1602460800000, 1602547200000, 1602633600000, 1602720000000, 1602806400000, 1602892800000, 1602979200000, 1603065600000, 1603152000000, 1603238400000, 1603324800000, 1603411200000, 1603497600000, 1603584000000, 1603670400000, 1603756800000, 1603843200000, 1603929600000, 1604016000000, 1604102400000, 1604188800000, 1604275200000, 1604361600000, 1604448000000, 1604534400000, 1604620800000, 1604707200000, 1604793600000, 1604880000000, 1604966400000, 1605052800000, 1605139200000, 1605225600000, 1605312000000, 1605398400000, 1605484800000, 1605571200000, 1605657600000, 1605744000000, 1605830400000, 1605916800000, 1606003200000, 1606089600000, 1606176000000, 1606262400000, 1606348800000, 1606435200000, 1606521600000, 1606608000000, 1606694400000, 1606780800000, 1606867200000, 1606953600000, 1607040000000, 1607126400000, 1607212800000, 1607299200000, 1607385600000, 1607472000000, 1607558400000, 1607644800000, 1607731200000, 1607817600000, 1607904000000, 1607990400000, 1608076800000, 1608163200000, 1608249600000, 1608336000000, 1608422400000, 1608508800000, 1608595200000, 1608681600000, 1608768000000, 1608854400000, 1608940800000, 1609027200000, 1609113600000, 1609200000000, 1609286400000, 1609372800000, 1609459200000, 1609545600000, 1609632000000, 1609718400000, 1609804800000, 1609891200000, 1609977600000, 1610064000000, 1610150400000, 1610236800000, 1610323200000, 1610409600000, 1610496000000, 1610582400000, 1610668800000, 1610755200000, 1610841600000, 1610928000000, 1611014400000, 1611100800000, 1611187200000, 1611273600000, 1611360000000, 1611446400000, 1611532800000, 1611619200000, 1611705600000, 1611792000000, 1611878400000, 1611964800000, 1612051200000, 1612137600000, 1612224000000, 1612310400000, 1612396800000, 1612483200000, 1612569600000, 1612656000000, 1612742400000, 1612828800000, 1612915200000, 1613001600000, 1613088000000, 1613174400000, 1613260800000, 1613347200000, 1613433600000, 1613520000000, 1613606400000, 1613692800000, 1613779200000, 1613865600000, 1613952000000, 1614038400000, 1614124800000, 1614211200000, 1614297600000, 1614384000000, 1614470400000, 1614556800000, 1614643200000, 1614729600000, 1614816000000, 1614902400000, 1614988800000]
N_MISSINGS = len(MISSINGS)
def df_to_arr_str(df):
    
    # convert df to str
    text = str(df.to_numpy())
    text = re.sub(' +', ' ', text)
    text = text.replace('\n ', ',').replace(' ', ',')
    
    # add missing dates
    text2 = '['
    text2 += ','.join([f'[{MISSINGS[i]},0]' for i in range(N_MISSINGS)]) + ','
    
    return text2 + text[1:]


# convert to js & highcharts format
df['timestamp'] = df['date'].astype('int64') // 10**6

# combine metadata & data
text_js = 'const vaccineData = {\n' \
          + '  src: "https://ourworldindata.org/covid-vaccinations?country=VNM",\n' \
          + '  partialPeople: ' + df_to_arr_str(df[['timestamp', 'people_vaccinated']])       + ',\n' \
          + '  fullPeople: '    + df_to_arr_str(df[['timestamp', 'people_fully_vaccinated']]) + ',\n' \
          + '  partialPct: '    + df_to_arr_str(df[['timestamp', 'people_vaccinated_pct']])   + ',\n' \
          + '  fullPct: '       + df_to_arr_str(df[['timestamp', 'people_fully_vaccinated_pct']]) + '\n'\
          + '};'

# save to file
with open("processed/vaccine-data.js", "w") as fo:
    fo.write(text_js)
np.set_printoptions(threshold=1000)
print('Vaccine data saved successfully')

Vaccine data saved successfully


# 4. Province Cases

In [9]:
def sort_prov(p_df, df, name_col):
    r_df = pd.DataFrame().reindex_like(df)  # result
    for i, core_name in enumerate(p_df['CoreName']):
        for j, name in enumerate(df[name_col]):
            if core_name in name:
                r_df.iloc[i] = df.iloc[j]
                break
    return r_df

def shift_col(df):
    cols = list(df.columns)
    return df[cols[-1:] + cols[:-1]]

In [10]:
# https://ncov.vncdc.gov.vn/viet-nam.html

# read dataframes
p_df = pd.read_excel('raw/provinces.xls')
df = pd.read_excel('raw/province-cases.xls', usecols=['Tên', 'Ca mắc', 'Tử vong'], dtype='str')
df.columns = ['Name', 'Cases', 'Deaths']

# sort province order as p_df
df = sort_prov(p_df, df, 'Name')

# process numeric columns
for col in ('Cases', 'Deaths'):
    df[col].fillna('0', inplace=True)
    df[col] = df.apply(lambda row: row[col].replace('.', ''), axis=1)
    df[col] = df[col].astype('int32')
    
# calculate other columns
df['CasesPer10Km2'] = df['Cases'] / p_df['Area']
df['ISO3166_2'] = p_df['ISO3166_2']
df['Name'] = p_df['Name']

df

,Name,Cases,Deaths,CasesPer10Km2,ISO3166_2
0,Hà Nội,4173,0,1.242482,VN-HN
1,Hồ Chí Minh,32902,12974,15.960997,VN-SG
2,Hà Giang,20,0,0.002522,VN-03
3,Cao Bằng,0,0,0.000000,VN-04
4,Bắc Kạn,6,0,0.001235,VN-53
...,...,...,...,...,...
58,Cần Thơ,5059,92,3.515636,VN-CT
59,Hậu Giang,489,2,0.301535,VN-73
60,Sóc Trăng,1021,25,0.308282,VN-52
61,Bạc Liêu,339,0,0.127014,VN-55


In [11]:
df.columns = ['vName', 'cases', 'deaths', 'value', 'iso3166_2']

# covert data to javascript object & json syntax
text_js = df.to_json(orient='records', force_ascii=False)
for col in df.columns:
    text_js = text_js.replace(f'"{col}"', col)
text_js = text_js.replace('\\', '\\\\')

# add metadata
text_js = 'const mapData = {src: "https://ncov.vncdc.gov.vn/viet-nam.html", updated: "2021-09-13 06:00", data: ' \
          + text_js + '};'

# save to file
with open("processed/map-data.js", "wb") as fo:
    fo.write(text_js.encode('utf8'))